In [ ]:
#   Copyright 2025 UKRI-STFC
#   Copyright 2025 University of Manchester

#   Licensed under the Apache License, Version 2.0 (the "License");
#   you may not use this file except in compliance with the License.
#   You may obtain a copy of the License at

#   http://www.apache.org/licenses/LICENSE-2.0

#   Unless required by applicable law or agreed to in writing, software
#   distributed under the License is distributed on an "AS IS" BASIS,
#   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#   See the License for the specific language governing permissions and
#   limitations under the License.
#
# Authors:
# Kubra Kumrular, University of Southampton, μ-VIS X-ray Imaging Centre 
# Evangelos Papoutsellis,  Finden Ltd
# Laura Murgatroyd (UKRI-STFC) 


In [ ]:
import numpy as np
import tifffile as tiff
import time
import os
import glob
import matplotlib.pyplot as plt
import multiprocessing
from cil.utilities.display import show2D, show_geometry

import xml.etree.ElementTree as ET
import threading


from cil.framework import AcquisitionGeometry, AcquisitionData
from cil.framework import ImageData, ImageGeometry
from cil.processors import Slicer, AbsorptionTransmissionConverter, TransmissionAbsorptionConverter,CentreOfRotationCorrector,Binner,Normaliser, Padder
from cil.utilities.display import show2D, show_geometry 
from cil.recon import FDK
from cil.plugins.tigre import FBP # check if FDK works with 2D

from cil.io import TIFFWriter, RAWFileWriter


import matplotlib as mpl

mpl.rcParams["image.cmap"] = "inferno"

### Diondo DataReader

Diondo data reader tested on 
 - Example raw data: https://zenodo.org/records/17167576
 - Recon: https://zenodo.org/records/17167562

1) Read centre slice and reconstruct (with/without COR)
2) Read all projections and reconstruct
3) Read specific angles


- TODOs
  - Check different range values CIL vs Diondo
  - Check strange line in sinogram 2D (maybe use
https://github.com/TomographicImaging/CIL-Demos/blob/flux_normaliser_demo/misc/flux_normaliser.ipynb)
  - Test the case of specific angles option
  - Check COR of CIL vs COR of Diondo
 



In [ ]:
from readers.DiondoDataReader import DiondoDataReader

### Path to data and metadata

In [ ]:
        
path    = r'20240508_d5_3917_AV_trustfireAAA_overv'
xml_file_name    = r'20240508_d5_3917_AV_trustfireAAA_overv.xml'
proj_file_name = r'20240508_d5_3917_AV_trustfireAAA_overv'


### Create DiondoDataReader

In [ ]:
data_reader = DiondoDataReader(xml_file=xml_file_name, 
                               projection_path=proj_file_name)

### Read Centre Slice (2D)

In [ ]:
t1 = time.time()
# no joblib
data2D = data_reader.read_centre_slice()
print(f" It took {time.time() - t1} sec for single slice")

### AcqGeom 2D info

In [ ]:
ag2D = data2D.geometry
ig2D = ag2D.get_ImageGeometry()
print(f"data2D dtype is {data2D.dtype}")
print(ag2D)

In [ ]:
show_geometry(ag2D)

### Recon 2D

In [ ]:
data_absorbed2D = TransmissionAbsorptionConverter(min_intensity=1e-6)(data2D)
data_absorbed2D -= np.mean(data_absorbed2D.array[0:50,0:50])
fdk_recon2D = FDK(data_absorbed2D, image_geometry=ig2D).run()

In [ ]:
data_absorbed2D = TransmissionAbsorptionConverter()(data2D)
data_absorbed2D -= np.mean(data_absorbed2D.array[0:10,0:10])
plt.imshow(data_absorbed2D.array)


In [ ]:
plt.imshow(fdk_recon2D.array)

### Correct CoR (in 2D centre slice)

In [ ]:
data_absorbed2D = CentreOfRotationCorrector.image_sharpness()(data_absorbed2D)

In [ ]:
fdk_recon2D_cor = FDK(data_absorbed2D, image_geometry=ig2D).run()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon2D.array)
axs[0].set_title('CIL without CoR', fontsize=20)

axs[1].imshow(fdk_recon2D_cor.array)
axs[1].set_title('CIL with CoR', fontsize=20)

plt.tight_layout()

plt.show()

### Read Full Data

### Read all projections/angles

In [ ]:
t1 = time.time()
data3D_all_fast = data_reader.read(fread=True)
print(f" With Joblib: {time.time() - t1} sec for {data3D_all_fast.shape[0]}")

t1 = time.time()
data3D_all_slow = data_reader.read(fread=False)
print(f" Without Joblib: {time.time() - t1} sec for {data3D_all_slow.shape[0]}")

### AcqGeom 3D info

In [ ]:
ag3D = data3D_all_fast.geometry
ig3D = ag3D.get_ImageGeometry()
print(f"data3D fast dtype is {data3D_all_fast.dtype}")
print(f"data3D slow dtype is {data3D_all_slow.dtype}")
print(ag3D)

In [ ]:
show_geometry(ag3D)

### Recon 3D with COR corr

In [ ]:
background      = data3D_all_fast.get_slice(vertical=10, force=True).as_array().mean()
data3D_norm          = data3D_all_fast/background
data3D_lb            = TransmissionAbsorptionConverter(min_intensity=1e-6)(data3D_norm)
data3D_no_outring = data3D_lb - np.mean(data3D_lb.array[0:100,:,0:50])
data3D_no_outring = CentreOfRotationCorrector.image_sharpness()(data3D_no_outring)

In [ ]:
print(data3D_no_outring.geometry)

In [ ]:
print(f"CIL offset after COR is -0.03750837")

tree = ET.parse(xml_file_name)
root = tree.getroot()
recon = root.find("Recon")

print(f"Diondo offset from metadata is {float(recon.find("ProjectionCenterOffsetX").text)}")


In [ ]:
fdk_recon3D_cor = FDK(data3D_no_outring, image_geometry=ig3D).run()

In [ ]:
plt.imshow(data3D_no_outring.array[1250//2])

In [ ]:
plt.imshow(fdk_recon3D_cor.array[1500])
plt.colorbar()

In [ ]:
with open(r"20240508_d5_3917_AV_trustfireAAA_overv_800x800x2270x32bit.raw", "rb") as f:
    tmp = np.fromfile(f, dtype="float32", count=800*800*2270)

In [ ]:
diondo_recon = tmp.reshape(2270,800,800)

In [ ]:
plt.imshow(fdk_recon3D_cor.array[1500])

In [ ]:
plt.imshow(diondo_recon[1738])

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon3D_cor.array[1500])
axs[0].set_title('CIL', fontsize=20)

axs[1].imshow(diondo_recon[1738])
axs[1].set_title('Diondo', fontsize=20)

plt.tight_layout()
# plt.subplots_adjust(wspace=-0.6, hspace=0.2)

# plt.savefig("cil_vs_zeiss_roi_recon.png")
plt.show()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

im0 = axs[0].imshow(fdk_recon3D_cor.array[1500, 400:500, 200:400])
axs[0].set_title('CIL', fontsize=20)
cbar0 = fig.colorbar(im0, ax=axs[0], shrink=0.9)

im1 = axs[1].imshow(diondo_recon[1738, 400:500, 200:400])
axs[1].set_title('Diondo', fontsize=20)
cbar1 = fig.colorbar(im1, ax=axs[1], shrink=0.9)

plt.tight_layout()
plt.show()


### Issue with different contrast, acqdata

In [ ]:
plt.imshow(data3D_all_fast.get_slice(vertical=0, force=True).as_array())
plt.colorbar()

In [ ]:
plt.imshow(np.mean(data3D_all_fast.array, axis=1))
plt.colorbar()

### Read some angles (user input) -> 625 out of 1250

In [ ]:
t1 = time.time()
data3D_some_angles1 = data_reader.read(angles = np.linspace(0, 360, 1250//2, endpoint=False), fread=True)
print(f" With Joblib: {time.time() - t1} sec for {data3D_some_angles1.shape[0]} projs")


In [ ]:
background      = data3D_some_angles1.get_slice(vertical=10, force=True).as_array().mean()
data3D_norm          = data3D_some_angles1/background
data3D_lb            = TransmissionAbsorptionConverter(min_intensity=1e-6)(data3D_norm)
data3D_no_outring = data3D_lb - np.mean(data3D_lb.array[0:100,:,0:50])
data3D_no_outring = CentreOfRotationCorrector.image_sharpness()(data3D_no_outring)



In [ ]:
fdk_recon1 = FDK(data3D_no_outring, image_geometry=ig3D).run()

In [ ]:
plt.imshow(fdk_recon1.array[1000], cmap="inferno")
plt.colorbar()

### Read some angles (user input) -> 312 out of 1250

In [ ]:
t1 = time.time()
data3D_some_angles2 = data_reader.read(angles = np.linspace(0, 360, 312, endpoint=False), fread=True)
print(f" With Joblib: {time.time() - t1} sec for {data3D_some_angles2.shape[0]} projs")


In [ ]:
background      = data3D_some_angles2.get_slice(vertical=10, force=True).as_array().mean()
data3D_norm          = data3D_some_angles2/background
data3D_lb            = TransmissionAbsorptionConverter(min_intensity=1e-6)(data3D_norm)
data3D_no_outring = data3D_lb - np.mean(data3D_lb.array[0:100,:,0:50])
data3D_no_outring = CentreOfRotationCorrector.image_sharpness()(data3D_no_outring)


In [ ]:
fdk_recon2 = FDK(data3D_no_outring, image_geometry=ig3D).run()

In [ ]:
plt.imshow(fdk_recon.array[1000], cmap="inferno")
plt.colorbar()

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon1.array[1000])
axs[0].set_title('CIL 625 angles', fontsize=20)

axs[1].imshow(fdk_recon2.array[1000])
axs[1].set_title('CIL 312 angles', fontsize=20)

plt.tight_layout()
# plt.subplots_adjust(wspace=-0.6, hspace=0.2)

# plt.savefig("cil_vs_zeiss_roi_recon.png")
plt.show()

In [ ]:
# ### test
# raw_files = sorted(glob.glob(os.path.join(proj_file_name, "*.raw")))

# print(len(raw_files)), 
# print(np.linspace(0, 360, 625, endpoint=False))

# # list_ang = np.linspace(0, 360, 1250, endpoint=False)
# list_ang = np.linspace(0, 360, 625, endpoint=False)
# print(len(list_ang))

# for i in range(0,len(raw_files)-1,2):
#     print(i, raw_files[i], list_ang[i])

### Test FluxNormaliser (old cil env)

In [ ]:
#  Copyright 2024 United Kingdom Research and Innovation
#  Copyright 2024 The University of Manchester
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
# Authors:
# CIL Developers, listed at: https://github.com/TomographicImaging/CIL/blob/master/NOTICE.txt

from cil.framework import Processor, AcquisitionData
from cil.utilities import multiprocessing as cil_mp
from cil.framework.labels import AcquisitionType

import numpy
import logging
import numba
import warnings

log = logging.getLogger(__name__)

class FluxNormaliser(Processor):
    r'''
    Flux normalisation based on float or region of interest

    This processor reads in an AcquisitionData and normalises it by flux from
    a float or array of float values, or the mean flux in a region of interest.
    Each projection is divided by its flux value and multiplied by the target.

    Parameters:
    -----------
    flux: float or list of floats, optional
        Array of floats that describe the variation in brightness of the unobstructed 
        beam between projections. Must have length equal to the number of projections 
        in the dataset, or be a single float. If flux=None, calculate
        flux from the roi.
    
    roi: dict, optional
        Dictionary describing the region of interest containing the background
        in the image from which to extract the flux. The roi is specified as 
        `{'horizontal':(start,stop), 'vertical':(start,stop)}`. If an axis is 
        not specified in the roi dictionary, the full range will be used.

    target: {'mean', 'first', 'last'} or float, default='mean'
        The target of the normalised data. If string the data is scaled by the 
        'mean', 'first' or 'last' flux value. If float, the data is scaled 
        by the float value.
        Default is 'mean'

    accelerated: bool, optional
        Specify whether to use multi-threading using numba. 
        Default is True

    Returns:
    --------
    Output: AcquisitionData normalised by flux

    Example
    -------
    This example passes the flux as a list the same size as the data, and 
    specifies the target='first' which scales all projections to the first flux
    value 0.9
    >>> from cil.processors import FluxNormaliser
    >>> processor = FluxNormaliser(flux=[0.9, 1.0, 1.1, 0.8], target='first')
    >>> processor.set_input(data)
    >>> data_norm = processor.get_output()

    Example
    -------
    This example calculates the flux from a region of interest for each projection
    and scales all projections to the mean flux
    >>> from cil.processors import FluxNormaliser
    >>> processor = FluxNormaliser(roi={'horizontal':(5, 15)}, target='mean')
    >>> processor.set_input(data)
    >>> data_norm = processor.get_output()

    Note
    ----
    The roi indices provided are start inclusive, stop exclusive.
    All elements along a dimension will be included if the axis does not appear 
    in the roi dictionary
    '''

    def __init__(self, flux=None, roi=None, target='mean', accelerated=True):
            
            kwargs = {
                    'flux'  : flux,
                    'roi' : roi,
                    'roi_slice' : None,
                    'roi_axes' : None,
                    'target' : target,
                    'target_value' : None,
                    'v_size' : 1,
                    'v_axis' : None,
                    'h_size' : 1,
                    'h_axis' : None,
                    '_accelerated' : accelerated
                    }
            super(FluxNormaliser, self).__init__(**kwargs)
            
    def check_input(self, dataset):

        if self.roi is not None and self.flux is not None:
            raise ValueError("Please specify either flux or roi, not both")
        if self.roi is None and self.flux is None:
            raise ValueError("Please specify either flux or roi, found None")
        
        if not (type(dataset), AcquisitionData):
            raise TypeError("Expected AcquistionData, found {}"
                            .format(type(dataset)))
        
        # if dataset.geometry.geom_type & AcquisitionType.CONE_FLEX:
            # raise NotImplementedError("FluxNormaliser does not yet support CONE3D_FLEX data")
        
        image_axes = 0
        if 'vertical' in dataset.dimension_labels:
            self.v_axis = dataset.get_dimension_axis('vertical')
            self.v_size = dataset.get_dimension_size('vertical')
            image_axes += 1

        if 'horizontal' in dataset.dimension_labels:
            self.h_axis = dataset.get_dimension_axis('horizontal')
            self.h_size = dataset.get_dimension_size('horizontal')
            image_axes += 1

        if (( self.h_axis is not None)  and (self.h_axis < (len(dataset.shape)-image_axes))) or \
            ((self.v_axis is not None) and self.v_axis < (len(dataset.shape)-image_axes)):
            raise ValueError('Projections must be the last two axes of the dataset')

        return True

    def _calculate_flux(self):
        '''
        Function to calculate flux from a region of interest in the data. If the 
        flux is already provided as an array, convert the array to float 32 and
        check the size matches the number of projections 
        '''

        dataset = self.get_input()
        if dataset is None:
            raise ValueError('Data not found, please run `set_input(data)`')
        
        # Calculate the flux from the roi in the data
        if self.flux is None:

            if isinstance(self.roi, dict):
                if not all (r in dataset.dimension_labels for r in self.roi):
                    raise ValueError("roi labels must be in the dataset dimension_labels, found {}"
                                    .format(str(self.roi)))

                slc = [slice(None)]*len(dataset.shape)
                axes=[]

                for r in self.roi:
                    # only allow roi to be specified in horizontal and vertical
                    if (r != 'horizontal' and r != 'vertical'):
                        raise ValueError("roi must be 'horizontal' or 'vertical', found '{}'"
                            .format(str(r)))
                    
                for d in ['horizontal', 'vertical']:
                    if d in self.roi:
                        # check indices are ints
                        if not all(isinstance(i, int) for i in self.roi[d]):
                            raise TypeError("roi values must be int, found {} and {}"
                            .format(str(type(self.roi[d][0])), str(type(self.roi[d][1]))))
                        # check indices are in range
                        elif (self.roi[d][0] >= self.roi[d][1]) or (self.roi[d][0] < 0) or self.roi[d][1] > dataset.get_dimension_size(d):
                            raise ValueError("roi values must be start > stop and between 0 and {}, found start={} and stop={} for direction '{}'"
                            .format(str(dataset.get_dimension_size(d)), str(self.roi[d][0]), str(self.roi[d][1]), d ))
                        # create slice
                        else:
                            ax = dataset.get_dimension_axis(d)
                            slc[ax] = slice(self.roi[d][0], self.roi[d][1])
                            axes.append(ax)
                    # if a projection dimension isn't in the roi, use the whole axis
                    else:
                        if d in dataset.dimension_labels:
                            ax = dataset.get_dimension_axis(d)
                            axes.append(ax)
                            self.roi.update({d:(0,dataset.get_dimension_size(d))})

                self.flux = numpy.mean(dataset.array[tuple(slc)], axis=tuple(axes))
                
                # Warn if the flux is more than 10% of the dataset range
                dataset_range = numpy.max(dataset.array, axis=tuple(axes)) - numpy.min(dataset.array, axis=tuple(axes)) 

                if (numpy.mean(self.flux) > dataset.mean()):
                    if numpy.mean(self.flux/dataset_range) < 0.9:
                        log.warning('Warning: mean value in selected roi is more than 10 percent of data range - may not represent the background')
                else:
                    if numpy.mean(self.flux/dataset_range) > 0.1:
                        log.warning('Warning: mean value in selected roi is more than 10 percent of data range - may not represent the background')

                self.roi_slice = slc
                self.roi_axes = axes
                
            else:
                raise TypeError("roi must be a dictionary, found {}"
                .format(str(type(self.roi))))
        
        # convert flux array to float32
        self.flux = numpy.array(self.flux, dtype=numpy.float32, ndmin=1)

        # check flux array is the right size
        flux_size_flat = len(self.flux.ravel())
        if flux_size_flat > 1:
            data_size_flat = dataset.geometry.num_projections*dataset.geometry.channels
            if data_size_flat != flux_size_flat:
                raise ValueError("Flux must be a scalar or array with length \
                                    \n = number of projections, found {} and {}"
                                    .format(flux_size_flat, data_size_flat))
          
    def _calculate_target(self):
        '''
        Calculate the target value for the normalisation
        '''

        if self.flux is None:
            raise ValueError('Flux not found')
            
        if isinstance(self.target, (int,float)):
            self.target_value = self.target
        elif isinstance(self.target, str):
            if self.target == 'first':
                if len(numpy.shape(self.flux)) > 0 :
                    self.target_value = self.flux.flat[0]
                else:
                    self.target_value = self.flux
            elif self.target == 'last':
                if len(numpy.shape(self.flux)) > 0 :
                    self.target_value = self.flux.flat[-1]
                else:
                    self.target_value = self.flux
            elif self.target == 'mean':
                self.target_value = numpy.mean(self.flux.ravel())
            else:
                raise ValueError("Target string not recognised, found {}, expected 'first' or 'mean'"
                                 .format(self.target))
        else:
            raise TypeError("Target must be string or a number, found {}"
                            .format(type(self.target)))
            
    def preview_configuration(self, angle=None, channel=None, log=False):
        '''
        Preview the FluxNormalisation processor configuration for roi mode.
        Plots the region of interest on the image and the mean, maximum and 
        minimum intensity in the roi.

        Requires matplotlib (or matplotlib-base) to be installed
        
        Parameters:
        -----------
        angle: float, optional
            Index of the angle to plot, default=None displays the data with the 
            minimum and maximum pixel values in the roi. For 2D data, the roi is 
            plotted on the sinogram.

        channel: int, optional
            The channel to plot, default=None displays the central channel if
            the data has channels

        log: bool, default=False
            If True, plot the image with a log scale, default is False

        Returns:
        --------
        matplotlib.figure.Figure
            The figure object created to plot the configuration
        '''
        import matplotlib.pyplot as plt

        self._calculate_flux()

        # check if flux array contains 0s
        if 0 in self.flux:
            warnings.warn('Flux value can\'t be 0, provide a different flux\
                                or region of interest with non-zero values')

        if self.roi_slice is None:
            raise ValueError('Preview available with roi, run `processor= FluxNormaliser(roi=roi)` then `set_input(data)`')
        else:
            
            data = self.get_input()

            min = numpy.min(data.array[tuple(self.roi_slice)], axis=tuple(self.roi_axes))
            max = numpy.max(data.array[tuple(self.roi_slice)], axis=tuple(self.roi_axes))
            if 'channel' in data.dimension_labels:
                if channel is None:
                    channel = int(data.get_dimension_size('channel')/2)
                channel_axis = data.get_dimension_axis('channel')
                flux_array = self.flux.take(indices=channel, axis=channel_axis)
                min = min.take(indices=channel, axis=channel_axis)
                max = max.take(indices=channel, axis=channel_axis)
            else:
                if channel is not None:
                    raise ValueError("Channel not found")
                else:
                    flux_array = self.flux
        
            plt.figure(figsize=(8,8))
            if data.geometry.dimension == '3D':
                if angle is None:
                    if 'angle' in data.dimension_labels:
                        self._plot_slice_roi(angle_index=numpy.argmin(min), channel_index=channel, log=log, ax=221)
                        self._plot_slice_roi(angle_index=numpy.argmax(max), channel_index=channel, log=log, ax=222)
                    else:
                        self._plot_slice_roi(log=log, channel_index=channel, ax=211)
                else:
                    if 'angle' in data.dimension_labels:
                        self._plot_slice_roi(angle_index=angle, channel_index=channel, log=log, ax=211)
                    else:
                        self._plot_slice_roi(log=log, channel_index=channel, ax=211)
                        
            # if data is 2D plot roi on all angles
            elif data.geometry.dimension == '2D':
                if angle is None:
                    self._plot_slice_roi(channel_index=channel, log=log, ax=211)
                else:
                    raise ValueError("Cannot plot ROI for a single angle on 2D data, please specify angle=None to plot ROI on the sinogram")
            
            plt.subplot(212)
            if data.geometry.num_projections==1:
                plt.plot(0, flux_array, '.r', label='Mean')
                plt.plot(0, min,'.k', label='Minimum')
                plt.plot(0, max,'.k', label='Maximum')
            else:
                indices = range(data.get_dimension_size('angle'))
                plt.plot(indices, flux_array, 'r', label='Mean')
                plt.plot(indices, min,'--k', label='Minimum')
                plt.plot(indices, max,'--k', label='Maximum')

            plt.legend()
            plt.xlabel('angle index')
            plt.ylabel('Intensity in roi')
            plt.grid()

            ax1 = plt.gca()
            ax2 = ax1.twiny()
            valid_ticks = [int(tick) for tick in ax1.get_xticks() if 0 <= tick < data.geometry.num_projections]
            ax2.set_xticks(valid_ticks)
            ax2.set_xbound(ax1.get_xbound())
            ax2.set_xticklabels([data.geometry.angles[tick] for tick in valid_ticks])
            ax2.set_xlabel('angle')
            
            plt.tight_layout()
            
            fig = plt.gcf()
            plt.show()

            return fig
            
    def _plot_slice_roi(self, angle_index=None, channel_index=None, log=False, ax=111):
        '''
        Plot the region of interest on a data slice
        Parameters:
        -----------
        angle_index: int, optional
            Index of the angle to plot
        channel_index: int, optional
            Index of the channel to plot
        log: bool, optional
            Plot the log of the slice intensity to highlight small variations
        ax: int, default=111
            The subplot axis to display the slice on
        '''
        import matplotlib.pyplot as plt
        
        data = self.get_input()
        if angle_index is not None and 'angle' in data.dimension_labels:
            data_slice = data.get_slice(angle=angle_index)
        else:
            data_slice = data
        
        if 'channel' in data.dimension_labels:
            data_slice = data_slice.get_slice(channel=channel_index)

        if len(data_slice.shape) != 2:
            raise ValueError("Data shape not compatible with preview_configuration(), data must have at least two of 'horizontal', 'vertical' and 'angle'")
        
        # if horizontal and vertical are not specified in the roi, get the
        # min and max extent from the full size of the dimension
        extent = [0, data_slice.shape[1], 0, data_slice.shape[0]]
        if 'angle' in data_slice.dimension_labels:
            min_angle = data_slice.geometry.angles[0]
            max_angle = data_slice.geometry.angles[-1]
            for i, d in enumerate(data_slice.dimension_labels):
                if d !='angle':
                    extent[i*2]=min_angle
                    extent[i*2+1]=max_angle

        # plot the specified data slice
        ax1 = plt.subplot(ax)
        if log:
            im = ax1.imshow(numpy.log(data_slice.array), cmap='gray',aspect='equal', origin='lower', extent=extent)
            plt.gcf().colorbar(im, ax=ax1)
        else:
            im = ax1.imshow(data_slice.array, cmap='gray',aspect='equal', origin='lower', extent=extent)
            plt.gcf().colorbar(im, ax=ax1)

        h = data_slice.dimension_labels[1]
        v = data_slice.dimension_labels[0]

        # get the box to plot from the roi
        if h == 'angle':
            h_min = min_angle
            h_max = max_angle
        else:
            h_min = self.roi[h][0]
            h_max = self.roi[h][1]

        if v == 'angle':
            v_min = min_angle
            v_max = max_angle
        else:
            v_min = self.roi[v][0]
            v_max = self.roi[v][1]

        # plot the roi box
        ax1.plot([h_min, h_max],[v_min, v_min],'--r')
        ax1.plot([h_min, h_max],[v_max, v_max],'--r')

        ax1.plot([h_min, h_min],[v_min, v_max],'--r')
        ax1.plot([h_max, h_max],[v_min, v_max],'--r')
        
        title = 'ROI'
        if angle_index is not None:
            title += ' angle = ' + str(data.geometry.angles[angle_index])
        if channel_index is not None:
            title += ' channel = ' + str(channel_index)
        ax1.set_title(title)

        ax1.set_xlabel(h)
        ax1.set_ylabel(v)
        
    def process(self, out=None):
        self._calculate_flux()
        if 0 in self.flux:
            raise ValueError('Flux value can\'t be 0, provide a different flux\
                                or region of interest with non-zero values')
        
        self._calculate_target()

        data = self.get_input()
        if out is None:
            out = data.copy()
        elif id(out) != id(data):
            numpy.copyto(out.array, data.array)

        proj_size = self.v_size*self.h_size
        num_proj = int(data.array.size / proj_size)
        if self._accelerated:
            num_threads_original = numba.get_num_threads()
            numba.set_num_threads(cil_mp.NUM_THREADS)
            numba_loop(self.flux, self.target_value, num_proj, proj_size, out.array)
            # reset the number of threads to the original value
            numba.set_num_threads(num_threads_original)
        else:
            serial_loop(self.flux, self.target_value, num_proj, proj_size, out.array)

        return out

@numba.njit(parallel=True)
def numba_loop(flux, target, num_proj, proj_size, out):
    out_flat = out.ravel()
    flux_flat = flux.ravel()
    if len(flux) == 1:
        norm = target/flux_flat[0]
        for i in numba.prange(num_proj):
            for ij in range(proj_size):
                out_flat[i*proj_size+ij] *= norm
    else:
        for i in numba.prange(num_proj):
            for ij in range(proj_size):
                out_flat[i*proj_size+ij] *= (target/flux_flat[i])

def serial_loop(flux, target, num_proj, proj_size, out):
    out_reshaped = out.reshape(num_proj, proj_size)
    flux_flat = flux.ravel() 
    norm = target / flux_flat[:, numpy.newaxis]  # shape: (num_proj, 1) 
    numpy.multiply(out_reshaped, norm, out=out_reshaped)

In [ ]:
processor = FluxNormaliser(roi={'horizontal':(0, 50)}, target='mean')
processor.set_input(data2D)
data_norm = processor.get_output()

In [ ]:
processor.preview_configuration()

In [ ]:
fig = plt.figure(figsize=(20,10))
plt.plot(np.mean(data_norm.array[:,0:50], axis=1), label="After Flux")
plt.plot(np.mean(data2D.array[:,0:50], axis=1), label="Before Flux")
plt.grid("on")

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(20, 10))

axs[0].imshow(fdk_recon1.array[1000])
axs[0].set_title('CIL 625 angles', fontsize=20)

axs[1].imshow(fdk_recon2.array[1000])
axs[1].set_title('CIL 312 angles', fontsize=20)

plt.tight_layout()
# plt.subplots_adjust(wspace=-0.6, hspace=0.2)

# plt.savefig("cil_vs_zeiss_roi_recon.png")
plt.show()

In [ ]:
plt.imshow(data3D.array)